# Streaming Results

Here is an example pattern if you want to stream your multiple results. Note that this is not supported for Hugging Face text completions at this time.


Initial configuration for the notebook to run properly.

In [6]:
# Make sure paths are correct for the imports

import os
import sys

notebook_dir = os.path.abspath("")
parent_dir = os.path.dirname(notebook_dir)
grandparent_dir = os.path.dirname(parent_dir)


sys.path.append(grandparent_dir)

We will load our settings and get the LLM service to use for the notebook.

In [7]:
from services import Service

from service_settings import ServiceSettings

service_settings = ServiceSettings.create()

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = (
    Service.AzureOpenAI
    if service_settings.global_llm_service is None
    else Service(service_settings.global_llm_service.lower())
)
print(f"Using service type: {selectedService}")

Using service type: Service.AzureOpenAI


First, we will set up the text and chat services we will be submitting prompts to.


In [8]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,  # noqa: F401
    AzureTextCompletion,
    OpenAIChatCompletion,
    OpenAIChatPromptExecutionSettings,  # noqa: F401
    OpenAITextCompletion,
    OpenAITextPromptExecutionSettings,  # noqa: F401
)
from semantic_kernel.contents import ChatHistory  # noqa: F401

kernel = Kernel()

service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"
    oai_chat_service = OpenAIChatCompletion(
        service_id="oai_chat",
        env_file_path="../.env",
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"
    aoai_chat_service = AzureChatCompletion(
        service_id="aoai_chat",
        env_file_path="../.env",
    )


Next, we'll set up the completion request settings for text completion services.


In [9]:
oai_prompt_execution_settings = OpenAITextPromptExecutionSettings(
    service_id="oai_text",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

Here, we're setting up the settings for Chat completions.


In [10]:
oai_chat_prompt_execution_settings = OpenAIChatPromptExecutionSettings(
    service_id="oai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

## Streaming Azure OpenAI Chat Completion


In [11]:
az_oai_chat_prompt_execution_settings = AzureChatPromptExecutionSettings(
    service_id="aoai_chat",
    max_tokens=150,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0.5,
    presence_penalty=0.5,
)

In [15]:
if selectedService == Service.AzureOpenAI:
    content = "You are an AI assistant that helps people find information."
    chat = ChatHistory()
    chat.add_system_message(content)
    chat.add_user_message("러버덕의 용도는 무엇인가요?")
    stream = aoai_chat_service.get_streaming_chat_message_contents(
        chat_history=chat, settings=az_oai_chat_prompt_execution_settings
    )
    async for text in stream:
        print(str(text[0]), end="")  # end = "" to avoid newlines

러버덕(Rubber Duck)은 주로 장난감으로 사용됩니다. 가장 일반적으로는 어린이들이 목욕 시간을 즐겁게 보내기 위해 목욕 장난감으로 사용됩니다. 또한, 러버덕은 프로그래밍과 소프트웨어 개발 분야에서 "러버덕 디버깅"이라는 용어로도 유명합니다. 이 기법에서는 개발자가 자신의 코드를 설명하기 위해 러버덕에게 말하는 것처럼 문제를 풀어내는 과정을 거칩니다. 이를 통해 문제를 더 명확하게 이해하고 해결 방법을 찾는 데 도움을 받을 수 있습니다.

러버덕은 상징성과 단순한 디자인 덕분에 다양한 문화적, 예술적